In [ ]:
import pandas as pd

In [ ]:
# 'Forex' 시트만 콕 집어서 불러오기
df = pd.read_excel('data/ForEx&IndexData.xls', sheet_name='Forex', index_col='Date')

# 결과 확인
print(df.head())

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
print(df.isnull().mean() * 100)

In [ ]:
df.columns

In [ ]:
means = df.mean()
means

In [ ]:
stds = df.std()
stds

In [ ]:
df = df.pct_change().dropna()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# 1. 기초 통계량 계산 (평균, 표준편차)
means = df.mean(numeric_only=True)
stds = df.std(numeric_only=True)
total_counts = df.count() # 각 열의 전체 데이터 개수

outlier_results = []

for col in df.columns:
    # 숫자형 데이터가 아닌 경우 건너뛰기
    if not pd.api.types.is_numeric_dtype(df[col]):
        continue
        
    # 상승/하락 개수 계산
    over_1std = ((df[col] - means[col]).abs() >= stds[col]).sum()
    over_2std = ((df[col] - means[col]).abs() >= 2 * stds[col]).sum()
    over_3std = ((df[col] - means[col]).abs() >= 3 * stds[col]).sum()
    over_4std = ((df[col] - means[col]).abs() >= 4 * stds[col]).sum()
    over_5std = ((df[col] - means[col]).abs() >= 5 * stds[col]).sum()
    over_6std = ((df[col] - means[col]).abs() >= 6 * stds[col]).sum()
    
    
    outlier_results.append({
        'Index': col,
        'Total': total_counts[col],
        'over 1 std': over_1std,
        'over 2 std': over_2std,
        'over 3 std': over_3std,
        'over 4 std': over_4std,
        'over 5 std': over_5std,
        'over 6 std': over_6std,
    })

# 결과 데이터프레임 생성
outlier_df = pd.DataFrame(outlier_results).set_index('Index')
print(outlier_df)

In [ ]:
outlier_df.T

In [ ]:
(outlier_df/999).T

In [ ]:
# 1. 'Forex' 시트만 콕 집어서 불러오기
df = pd.read_excel('data/ForEx&IndexData.xls', sheet_name='Indices')

# 2. 'Date' 컬럼을 날짜 데이터 타입으로 변환 (중요!)
# '27-Mar-06' 같은 형식을 인식하도록 설정합니다.
# df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

# 3. 날짜를 인덱스로 설정 
# df.set_index('Date')

# 결과 확인
print(df.head())

In [ ]:
names = [df.columns[i] for i in range(1, 8, 2)]
names

In [ ]:
df = df[names]
df

In [ ]:
df = df.pct_change().dropna()
df

In [ ]:
# 1. 기초 통계량 계산 (평균, 표준편차)
means = df.mean(numeric_only=True)
stds = df.std(numeric_only=True)
total_counts = df.count() # 각 열의 전체 데이터 개수

outlier_results = []

for col in df.columns:
    # 숫자형 데이터가 아닌 경우 건너뛰기
    if not pd.api.types.is_numeric_dtype(df[col]):
        continue
        
    # 상한선(Upper)과 하한선(Lower) 설정
    upper_limit = means[col] + (3 * stds[col])
    lower_limit = means[col] - (3 * stds[col])
    
    # 상승/하락 개수 계산
    count_upper = (df[col] > upper_limit).sum()
    count_lower = (df[col] < lower_limit).sum()
    
    # 비율 계산 (%)
    pct_upper = (count_upper / total_counts[col]) * 100
    pct_lower = (count_lower / total_counts[col]) * 100
    
    outlier_results.append({
        'Index': col,
        'Total': total_counts[col],
        'High Outliers (>3std)': count_upper,
        'High %': pct_upper.round(2),
        'Low Outliers (<3std)': count_lower,
        'Low %': pct_lower.round(2)
    })

# 결과 데이터프레임 생성
outlier_df = pd.DataFrame(outlier_results).set_index('Index')
print(outlier_df)

In [ ]:
import matplotlib.pyplot as plt

# 비율(%) 데이터만 추출해서 시각화
outlier_df[['High %', 'Low %']].plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Extreme Movements (Over 3*Std)')
plt.ylabel('Percentage of Total Data (%)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import bsm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
help(bsm.get_price)

In [ ]:
S, r, q, T = 1.0, 0.025, 0.025, 0.5

quotes = [
    (0.7, 0.13),
    (0.75, 0.125),
    (0.8, 0.12),
    (0.85, 0.115),
    (0.9, 0.11),
    (0.95, 0.105),
    (1.0, 0.10),
    (1.05, 0.105),
    (1.1, 0.11),
    (1.15, 0.115),
    (1.2, 0.12),
    (1.25, 0.125),
    (1.3, 0.13)
]

smile_price = {k: bsm.get_price(S=S, K=k, r=r, T=T, q=q, vol=vol) for k, vol in quotes}
smile_price

In [ ]:
constant_price = {k: bsm.get_price(S=S, K=k, r=r, T=T, q=q, vol=0.115) for k, v in quotes}
constant_price

In [ ]:
# 1. Series 변환 (Key 기준 정렬 보장)
s = pd.Series(ss).sort_index()

# 2. rolling 연산 수행
# window=3으로 묶고 커스텀 함수 apply

def approx(series, h=0.05):
    return series.rolling(window=3, center=True).apply(
        lambda x: np.exp(r * T) * (x.iloc[0] + x.iloc[2] - 2 * x.iloc[1]) / (h**2)    
    )
smile_graph = approx(pd.Series(smile_price)).iloc[1::2]   
constant_graph = approx(pd.Series(constant_price)).iloc[1::2]
constant_graph

In [ ]:
1/5*(12.82)+4/5*(13.7)

In [ ]:
edges = np.arange(0.75, 1.35, 0.1)
print(edges)

plt.figure(figsize=(10, 6))

# where='post' 옵션으로 각 밀도값이 해당 구간 동안 유지됨을 표현
plt.step(edges[:-1], smile_graph, where='mid', color='red', label='Market (Smile)', linewidth=2)
plt.step(edges[:-1], constant_graph, where='mid', color='blue', linestyle='--', label='Constant Vol (11.5%)', linewidth=2)

plt.title('Implied Probability Density Function (Step-wise)', fontsize=14)
plt.xlabel('Strike Price (K)', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xticks(ks)
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.7)
plt.show()